In [3]:
#  imports
import os
import sys
import time
from datetime import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import mne

In [4]:
data_path = "/Users/rufaelmarew/Documents/tau/finger_pose_estimation/dataset/FPE/S1/p3/fpe_pos3_001_S1_rep0_BT.edf"


In [65]:
def read_emg(path, start_time=None, end_time=None, fs: int=250):

    raw = mne.io.read_raw_edf(path, preload=True)

    # get header
    header = raw.info

    if start_time is None:
        start_time = header['meas_date']
        # convert to pd.datetime from datetime.datetime
        start_time = pd.to_datetime(start_time).tz_localize(None)
        print(start_time)
    #  get annotations
    annotations = raw.annotations
    annotations.onset = start_time + pd.to_timedelta(annotations.onset, unit='s')
    
    # to dataframe
    emg_df = raw.to_data_frame()


    emg_df['time'] = pd.to_datetime(emg_df['time'], unit='s', utc=True, origin=start_time)
    emg_df.set_index('time', inplace=True)

    #  resample emg data to fs Hz
    emg_df = emg_df.resample(f'{int(1000/fs)}ms', origin='start').mean()
    return emg_df, annotations, header

In [67]:
emg_data, ann, header = read_emg(path = data_path)

Extracting EDF parameters from /Users/rufaelmarew/Documents/tau/finger_pose_estimation/dataset/FPE/S1/p3/fpe_pos3_001_S1_rep0_BT.edf...
EDF file detected
Setting channel info structure...
Creating raw.info structure...
Reading 0 ... 135499  =      0.000 ...   541.996 secs...
2024-01-02 13:17:25
2024-01-02 13:17:29.048000 start_abduction
2024-01-02 13:17:56.096000 end_abduction
2024-01-02 13:18:01.136000 start_threefingers
2024-01-02 13:18:06.172000 end_threefingers
2024-01-02 13:18:11.288000 start_threefingers
2024-01-02 13:18:16.244000 end_threefingers
2024-01-02 13:18:21.436000 start_tet
2024-01-02 13:18:26.392000 end_tet
2024-01-02 13:18:31.516000 start_twofingers
2024-01-02 13:18:36.552000 end_twofingers
2024-01-02 13:18:41.588000 start_threefingers
2024-01-02 13:18:46.628000 end_threefingers
2024-01-02 13:18:51.744000 start_fist
2024-01-02 13:18:56.700000 end_fist
2024-01-02 13:19:01.812000 start_het
2024-01-02 13:19:06.852000 end_het
2024-01-02 13:19:11.920000 start_abduction
202

In [71]:
def create_windowed_dataset(df, w, s, fs):
    # Convert window size and stride from seconds to number of rows
    w_rows = int(w * fs)
    s_rows = int(s * fs)
    print(w_rows, s_rows)
    data = []
    times = []
    for i in range(0, len(df) - w_rows, s_rows):
        window = df.iloc[i:i+w_rows]
        data.append(window.values)
        times.append(window.index[-1])

    data = np.array(data)
    times = np.array(times)

    # Reshape data to (N-w)/(S)*W*C
    data = data.reshape((-1, w_rows * df.shape[1]))

    return data, times

data, times = create_windowed_dataset(emg_data, 1, 0.2, fs=250)
data.shape, times.shape

25000 50


((2210, 400000), (2210,))